In [1]:
import pandas as pd
import numpy as np
import tarfile
%reload_ext version_information
np.__version__, pd.__version__

('1.16.4', '0.24.2')

<h3>2a) Einlesen und Entpacken des tar.gz Files mit tarfile</h3>

In [7]:
tar = tarfile.open("20news-18828.tar.gz")
#tar.extractall() #gefährlich, entpackt direkt alles
#tar.close()

<h3> 2b) Speichern der Beiträge der Newsgroups <I>alt.atheism, comp.graphics, sci.space </I>
    und <I>talk.religion.misc</I> als Strings in ein Array</h3>

In [8]:
i = 0
for tarinfo in tar:
    print(tarinfo.name, "is", tarinfo.size, "bytes in size and is ", end="")
    if tarinfo.isreg():
        print("a regular file.")
    elif tarinfo.isdir():
        print("a directory.")
    else:
        print("something else.")
        
    i+=1
    if i == 20:
        break


20news-18828 is 0 bytes in size and is a directory.
20news-18828/alt.atheism is 0 bytes in size and is a directory.
20news-18828/alt.atheism/51203 is 1372 bytes in size and is a regular file.
20news-18828/alt.atheism/51277 is 1514 bytes in size and is a regular file.
20news-18828/alt.atheism/53192 is 1752 bytes in size and is a regular file.
20news-18828/alt.atheism/53222 is 1991 bytes in size and is a regular file.
20news-18828/alt.atheism/51283 is 2615 bytes in size and is a regular file.
20news-18828/alt.atheism/53759 is 577 bytes in size and is a regular file.
20news-18828/alt.atheism/53225 is 753 bytes in size and is a regular file.
20news-18828/alt.atheism/53098 is 1200 bytes in size and is a regular file.
20news-18828/alt.atheism/51275 is 2386 bytes in size and is a regular file.
20news-18828/alt.atheism/53601 is 892 bytes in size and is a regular file.
20news-18828/alt.atheism/54173 is 1587 bytes in size and is a regular file.
20news-18828/alt.atheism/53334 is 1045 bytes in siz

In [27]:
i = 0
groups = ["20news-18828/alt.atheism", "20news-18828/comp.graphics", "20news-18828/sci.space", "20news-18828/talk.religion.misc"]
data = []
needed = False
for tarinfo in tar:
    if tarinfo.isdir() and tarinfo.name in groups:
        needed = True
    elif tarinfo.isreg() and needed:
        data.append(open(tarinfo.name).read())
    elif tarinfo.isdir() and tarinfo.name not in groups:
        needed = False

Kontrolle: Es müssten jetzt 3387 Strings
im Speicher sein

In [28]:
np.shape(data)

(3387,)

Entfernen der ersten zwei Zeilen, also dem Newsgroup-Header, aus den Dateien

In [30]:
def strip_header(text):
    _before, _blankline, after = text.partition ('\n\n')
    return after


In [31]:
data = [strip_header(text) for text in data]

<h3>2c) </h3>

In [2]:
%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.6.1
OS Windows 10 10.0.18362 SP0
numpy 1.16.4
pandas 0.24.2
Tue Nov 12 12:31:02 2019 Mitteleuropäische Zeit